## Jupiter Dependency Check
 Esse notebook irá pegar as tabelas dos sites que contêm a grade curricular do BCC, e as relacionar com seus oferecimentos nos últimos 4 anos. Além disso, também irá verificar se os requisitos de cada disciplina não estão sendo oferecidos em semestres posteriores às que dependem destes.

## Declaração e importação de libs

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from unicodedata import normalize
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(
    executable_path=r'C:/Program Files/webdrivers/chromedriver.exe')

<ipython-input-1-71e05f28a057>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


# Pegando a grade curricular e os oferecimentos nos últimos 4 anos.

In [2]:

gradeCurricular = pd.read_html('https://uspdigital.usp.br/jupiterweb/listarGradeCurricular?codcg=45&codcur=45052&codhab=1&tipo=N',
                        match= "Disciplinas");

# [0] = 1° ano; [1] = 2° ano; [3] = 3° e 4° anos

segundoSemestre2021 = [0]*3;
primeiroSemestre2021 = [0]*3;
segundoSemestre2020 = [0]*3;
primeiroSemestre2020 = [0]*3;
segundoSemestre2019 = [0]*3;
primeiroSemestre2019 = [0]*3;


driver.get('https://www.ime.usp.br/~pmiranda/horarios/2s21/index.html')
segundoSemestre2021[0]=pd.read_html(driver.page_source)[0]
segundoSemestre2021[1]=pd.read_html(driver.page_source)[1]
segundoSemestre2021[2]=pd.read_html(driver.page_source)[2]
driver.quit

driver.get('https://www.ime.usp.br/~pmiranda/horarios/1s21/index.html')
primeiroSemestre2021[0]=pd.read_html(driver.page_source)[0]
primeiroSemestre2021[1]=pd.read_html(driver.page_source)[1]
primeiroSemestre2021[2]=pd.read_html(driver.page_source)[2]
driver.quit

driver.get('https://www.ime.usp.br/~pmiranda/horarios/2s20/index.html')
segundoSemestre2020[0]=pd.read_html(driver.page_source)[0]
segundoSemestre2020[1]=pd.read_html(driver.page_source)[1]
segundoSemestre2020[2]=pd.read_html(driver.page_source)[2]
driver.quit

driver.get('https://www.ime.usp.br/~pmiranda/horarios/1s20/index.html')
primeiroSemestre2020[0]=pd.read_html(driver.page_source)[0]
primeiroSemestre2020[1]=pd.read_html(driver.page_source)[1]
primeiroSemestre2020[2]=pd.read_html(driver.page_source)[2]
driver.quit

driver.get('https://www.ime.usp.br/~pmiranda/horarios/2s19/index.html')
segundoSemestre2019[0]=pd.read_html(driver.page_source)[0]
segundoSemestre2019[1]=pd.read_html(driver.page_source)[1]
segundoSemestre2019[2]=pd.read_html(driver.page_source)[2]
driver.quit

driver.get('https://www.ime.usp.br/~pmiranda/horarios/1s19/index.html')
primeiroSemestre2019[0]=pd.read_html(driver.page_source)[0]
primeiroSemestre2019[1]=pd.read_html(driver.page_source)[1]
primeiroSemestre2019[2]=pd.read_html(driver.page_source)[2]
driver.quit

<bound method ChromiumDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="15fc24ea8ef15afd9cdc9250a44454aa")>>

## Pegando os códigos das disciplinas sendo oferecidas:

In [3]:
df = gradeCurricular[3];

In [4]:
codigos = {};
for i in df[0]:
    if(isinstance(i, str)):
        if(re.match("^[0-9]", i)):
            x = i.split()[0][0].split()
            y = x[0];
            
        if(re.match("^(([A-Z]{3})[0-9]{4})$", i)):
            codigos[i] = [float(y)]; 

## Pegando os requisitos de cada disciplina:

In [5]:
for cod in codigos:   
    requisitosDisciplina = pd.read_html("https://uspdigital.usp.br/jupiterweb/listarCursosRequisitos?coddis="+cod);
    if(len(requisitosDisciplina) >= 6):
        req = requisitosDisciplina[6];
        for i in req:
            j = 1;
            k = 0;
            try:
                while(((re.match("(^(Curso:\ 45051))(.*)", req[i][k]) == None) and
                        (re.match("(^(Curso:\ 45052))(.*)", req[i][k]) == None)) and len(req[i])> k):
                    k+=1;
                    while(isinstance(req[i][k], str) == False): k+=1;
                if(re.match("(^(Curso:\ 45051))(.*)", req[i][k]) or
                   (re.match("(^(Curso:\ 45052))(.*)", req[i][k]))):
                    while(isinstance(req[i][k+j], str)):
                        repetida = 0;
                        if(re.match("(^(([A-Z]{3})[0-9]{4}))(.*)", req[i][k+j])):
                            for z in codigos[cod]:
                                if z == req[i][k+j].split(' ')[0]:
                                    repetida = 1;
                                    break;
                            if repetida == 0:    
                                codigos[cod].append(req[i][k+j].split(' ')[0]);
                        j+=1;
            except:
                print("Essa disciplina não possui requisitos para o BCC:", cod);


Essa disciplina não possui requisitos para o BCC: ACH2023
Essa disciplina não possui requisitos para o BCC: ACH2023
Essa disciplina não possui requisitos para o BCC: ACH2023
Essa disciplina não possui requisitos para o BCC: ACH2023
Essa disciplina não possui requisitos para o BCC: ACH2023
Essa disciplina não possui requisitos para o BCC: ACH2033
Essa disciplina não possui requisitos para o BCC: ACH2033
Essa disciplina não possui requisitos para o BCC: ACH2033
Essa disciplina não possui requisitos para o BCC: ACH2033
Essa disciplina não possui requisitos para o BCC: ACH2033
Essa disciplina não possui requisitos para o BCC: QBQ0126
Essa disciplina não possui requisitos para o BCC: QBQ0126
Essa disciplina não possui requisitos para o BCC: QBQ0126
Essa disciplina não possui requisitos para o BCC: QBQ0126
Essa disciplina não possui requisitos para o BCC: QBQ0126
Essa disciplina não possui requisitos para o BCC: QBQ1252
Essa disciplina não possui requisitos para o BCC: QBQ1252
Essa disciplin

Essa disciplina não possui requisitos para o BCC: ACH2028
Essa disciplina não possui requisitos para o BCC: ACH2028
Essa disciplina não possui requisitos para o BCC: ACH2028
Essa disciplina não possui requisitos para o BCC: ACH2028
Essa disciplina não possui requisitos para o BCC: ACH2028
Essa disciplina não possui requisitos para o BCC: ACH2037
Essa disciplina não possui requisitos para o BCC: ACH2037
Essa disciplina não possui requisitos para o BCC: ACH2037
Essa disciplina não possui requisitos para o BCC: ACH2037
Essa disciplina não possui requisitos para o BCC: ACH2037
Essa disciplina não possui requisitos para o BCC: ACH2038
Essa disciplina não possui requisitos para o BCC: ACH2038
Essa disciplina não possui requisitos para o BCC: ACH2038
Essa disciplina não possui requisitos para o BCC: ACH2038
Essa disciplina não possui requisitos para o BCC: ACH2038
Essa disciplina não possui requisitos para o BCC: ACH2048
Essa disciplina não possui requisitos para o BCC: ACH2048
Essa disciplin

In [6]:
print(codigos)

{'MAC0101': [1.0], 'MAC0105': [1.0], 'MAC0110': [1.0], 'MAC0329': [1.0], 'MAT0112': [1.0], 'MAT2453': [1.0], 'MAC0121': [2.0, 'MAC0110'], 'MAC0216': [2.0, 'MAC0110'], 'MAC0239': [2.0, 'MAC0110'], 'MAE0119': [2.0], 'MAT0122': [2.0, 'MAT0112'], 'MAT2454': [2.0, 'MAT2453'], 'MAC0102': [3.0, 'MAC0121'], 'MAC0209': [3.0, 'MAC0110'], 'MAC0210': [3.0, 'MAC0110', 'MAT0122'], 'MAC0323': [3.0, 'MAC0121'], 'MAT0236': [3.0, 'MAT2454'], 'MAC0316': [4.0, 'MAC0121'], 'MAC0338': [4.0, 'MAC0323'], 'MAC0350': [5.0, 'MAC0316'], 'MAC0422': [5.0, 'MAC0121'], 'FLC0474': [7.0], 'MAC0499': [7.0, 'MAC0338', 'MAC0422'], 'GMG0630': [1.0], 'ACH2023': [2.0], 'ACH2033': [2.0], 'MAC0213': [2.0], 'MAC0214': [2.0], 'MAC0215': [2.0], 'CJE0642': [3.0], 'IOF0115': [3.0, 'MAC0110'], 'MAC0327': [3.0, 'MAC0121'], 'MAC0337': [3.0, 'MAC0121'], 'MAC0453': [3.0], 'MAE0217': [3.0, 'MAE0119'], 'MAE0228': [3.0, 'MAE0212'], 'MAE0399': [3.0, 'MAE0119'], 'MAP2001': [3.0], 'MAT0206': [3.0, 'MAT2454'], 'MAT0222': [3.0, 'MAT0122'], 'MAT

# Verifica se as disciplinas estão sendo oferecidos nos semestres corretos

In [ ]:
for codi in codigos:
    
    if(codigos[codi][0]%2.0 == 0):
        
        try:
            
            segundoSemestre2021[0].loc(codi);
            segundoSemestre2021[1].loc(codi);
            segundoSemestre2021[2].loc(codi);
            segundoSemestre2020[0].loc(codi);
            segundoSemestre2020[1].loc(codi);
            segundoSemestre2020[2].loc(codi);
            segundoSemestre2019[0].loc(codi);
            segundoSemestre2019[1].loc(codi);
            segundoSemestre2019[2].loc(codi);
            
    elif(codigos[codi][0] % 2.0 != 0):
        
        try:
            
            primeiroSemestre2021[0].loc(codi);
            primeiroSemestre2021[1].loc(codi);
            primeiroSemestre2021[2].loc(codi);
            primeiroSemestre2020[0].loc(codi);
            primeiroSemestre2020[1].loc(codi);
            primeiroSemestre2020[2].loc(codi);
            primeiroSemestre2019[0].loc(codi);
            primeiroSemestre2019[1].loc(codi);
            primeiroSemestre2019[2].loc(codi);
